In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf

from baseline import Baseline
from system_t import System_T

# Load Fashion-MNIST dataset from keras

In [2]:
fashion = tf.keras.datasets.fashion_mnist
(fashion_train_images,fashion_train_labels), (fashion_test_images, fashion_test_labels) = fashion.load_data()

fashion_train_labels = fashion_train_labels
fashion_test_labels = fashion_test_labels
fashion_train_images = fashion_train_images.reshape((60000, 28, 28, 1)) / 255.0
fashion_test_images = fashion_test_images.reshape((10000, 28, 28, 1)) / 255.0

train_images = fashion_train_images
train_labels = fashion_train_labels

# Basic setting: slices have the same amounts of data

In [3]:
def shuffle(data, label):
    shuffle = np.arange(len(data))
    np.random.shuffle(shuffle)
    data = data[shuffle]
    label = label[shuffle]
    return data, label

num_class = len(np.unique(train_labels))
print("Number of slices : %d" % num_class)

y_train_one_hot = tf.keras.utils.to_categorical(train_labels)
mixed_data = (train_images, y_train_one_hot)

initial_data_array = []
val_data_dict = []
add_data_dict = []
    
val_data_num = 500

for i in range(num_class):
    data_num = 100
    initial_data_array.append(data_num)
    idx = np.argmax(mixed_data[1], axis=1) == i

    val_data_dict.append((mixed_data[0][idx][data_num:data_num+val_data_num], mixed_data[1][idx][data_num:data_num+val_data_num]))
    add_data_dict.append((mixed_data[0][idx][data_num+val_data_num:], mixed_data[1][idx][data_num+val_data_num:]))

    if i == 0:
        train_data = mixed_data[0][idx][:data_num]
        train_label = mixed_data[1][idx][:data_num]
        val_data = mixed_data[0][idx][data_num:data_num+val_data_num]
        val_label = mixed_data[1][idx][data_num:data_num+val_data_num]
    else:
        train_data = np.concatenate((train_data, mixed_data[0][idx][:data_num]), axis=0)
        train_label = np.concatenate((train_label, mixed_data[1][idx][:data_num]), axis=0) 
        val_data = np.concatenate((val_data, mixed_data[0][idx][data_num:data_num+val_data_num]), axis=0)
        val_label = np.concatenate((val_label, mixed_data[1][idx][data_num:data_num+val_data_num]), axis=0)   
    
train_data, train_label = shuffle(train_data, train_label)
print(train_data.shape, train_label.shape, num_class)

Number of slices : 10
(1000, 28, 28, 1) (1000, 10) 10


# Define slices

In [4]:
slice_desc = []
a = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

for i in range(num_class):
    slice_desc.append('Slice: %s' % (a[i]))
    print('Slice: %s, Initial size: %s' % (a[i], initial_data_array[i]))

Slice: T-shirt/top, Initial size: 100
Slice: Trouser, Initial size: 100
Slice: Pullover, Initial size: 100
Slice: Dress, Initial size: 100
Slice: Coat, Initial size: 100
Slice: Sandal, Initial size: 100
Slice: Shirt, Initial size: 100
Slice: Sneaker, Initial size: 100
Slice: Bag, Initial size: 100
Slice: Ankle boot, Initial size: 100


# Original ( with no data acquisition ) 

In [5]:
cost_func = [1] * num_class
lr = 0.0005

ori = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
ori.performance(budget=0, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 0
======= Collect Data =======
[0 0 0 0 0 0 0 0 0 0]
======= Performance =======
Loss: 0.47202 (0.00337), Average EER: 0.28311 (0.01103), Max EER: 0.69305 (0.12473)



# System T Demo on Fashion-MNIST

## Use 3000 budget, lambda=0.1, "Moderate" strategy

In [6]:
budget = 3000
method = 'Moderate'

st = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, add_data_dict)
st.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=2000, cost_func=cost_func, 
                 Lambda=0.1, num_iter=5, slice_desc=slice_desc, strategy=method, show_figure=False)

======= Collect Data =======
[ 31  19  45  53  73  25 100  38  11   0]
Total Cost: 395, Remaining Budget: 2605
======= Collect Data =======
[  0  20 225 104 218  64 292  50  91  23]
Total Cost: 1087, Remaining Budget: 1518
======= Collect Data =======
[524  80 204 114 318  43  97 138   0   0]
Total Cost: 1518, Remaining Budget: 0

======= Performance =======
[555. 119. 474. 271. 609. 132. 489. 226. 102.  23.]
Number of iteration: 3
Strategy: Moderate, C: 0.1, Budget: 3000
Loss: 0.34054 (0.00440), Average EER: 0.17254 (0.01276), Max EER: 0.47345 (0.05491)



# Baseline: Uniform ( = Water filling )

## For a basic setting, Uniform method is equivalent to Water filling method

In [7]:
budget = 3000
uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 3000
======= Collect Data =======
[300 300 300 300 300 300 300 300 300 300]
======= Performance =======
Loss: 0.35459 (0.00572), Average EER: 0.23012 (0.00799), Max EER: 0.69736 (0.07020)



# Summary of results

| Method | Loss | Avg.EER |
|:---------------------:|:---------------------:|:---------------------:|
| Original | 0.47202 (± 0.00169) | 0.28311 (± 0.00551) |
| Uniform | 0.35459 (± 0.00286) | 0.23012 (± 0.00400) |
| Water filling | 0.35459 (± 0.00286) | 0.23012 (± 0.00400) |
| Moderate (ours) | 0.34054 (± 0.00220) | 0.17254 (± 0.00638) |